<a href="https://colab.research.google.com/github/pouyan6/ireal/blob/main/iReal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall clip -y
!pip install git+https://github.com/openai/CLIP.git

Found existing installation: clip 1.0
Uninstalling clip-1.0:
  Successfully uninstalled clip-1.0
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-8d6pw7yn
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-8d6pw7yn
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=c5cdfbde10de927ad9b2be5f9a0877e31b05682e895a4cf89ef481e3797f7cbd
  Stored in directory: /tmp/pip-ephem-wheel-cache-3lc7e9xd/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [ ]:
!pip install ftfy regex tqdm

# Import libs

In [ ]:
import torch
import clip
from PIL import Image

# Load Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 140MiB/s]


# Load two images

In [ ]:
img1 = preprocess(Image.open("house-1.jpg")).unsqueeze(0).to(device)
img2 = preprocess(Image.open("house-1.jpg")).unsqueeze(0).to(device)


FileNotFoundError: [Errno 2] No such file or directory: 'house-1.jpg'